In [1]:
import pandas as pd

In [2]:
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)

In [3]:
caesar_male = man_df[man_df['sex'] == 1]

In [4]:
data=pd.DataFrame()

In [5]:
data=caesar_male.copy()

In [6]:
data.describe()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_ventre_maximal,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,...,angle_de_l_epaule,longueur_du_côte,longueur_du_tronc,longueur_de_sous_poitrine,largeur_de_la_taille,tour_de_poignet,largeur_du_bassin,profondeur_de_poitrine,tour_d_epaule,longueur_d_avant_bras
count,2106.000000,2102.000000,2106.000000,2107.0,2107.000000,2104.000000,2107.0,2107.0,2106.000000,0.0,...,2107.0,2107.0,2107.0,2107.0,2107.0,2107.0,2107.0,2107.0,2107.0,2052.000000
mean,177.913588,37.538297,82.985266,1.0,26.720221,64.139418,0.0,0.0,101.697680,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.407886
std,8.431030,12.763015,17.167743,0.0,1.545217,3.591747,0.0,0.0,10.616289,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.615831
min,131.495800,17.500000,45.812792,1.0,22.199600,40.894000,0.0,0.0,75.793600,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.154900
25%,172.212000,26.200000,70.801175,1.0,25.704800,61.798200,0.0,0.0,94.411800,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.311100
50%,177.495200,37.000000,80.058988,1.0,26.593800,64.008000,0.0,0.0,100.406200,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.320750
75%,183.311800,47.000000,91.398788,1.0,27.711400,66.395600,0.0,0.0,107.670600,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.432000
max,218.287600,79.000000,181.436800,1.0,36.703000,80.391000,0.0,0.0,157.403800,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.264600


In [7]:
data=data.drop_duplicates()

In [8]:
mesure=caesar_male[['tour_de_ventre_maximal','tour_de_taille','tour_de_cuisse','largeur_des_hanches']]

In [9]:
data["tour_de_cuisse_ratio"] = data["tour_de_cuisse"] / data["taille"]

In [10]:
data["tour_de_taille_ratio"] = data["tour_de_taille"] / data["taille"]

In [11]:
data["largeur_d_epaule_ratio"] = data["largeur_d_epaule"] / data["taille"]

In [12]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if tour_cuisse < 0.312557:
        cuisses = "fines"
    elif  0.312557 <=tour_cuisse<0.350709 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
data[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = data.apply(classifier_morphologie, axis=1)

In [13]:
data=data.loc[:, (data != 0).any()]

In [14]:
data = data.dropna(how='all', axis=1)

In [15]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

data = remplacer_nan_par_moyenne(data)
data.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,...,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras,tour_de_cuisse_ratio,tour_de_taille_ratio,largeur_d_epaule_ratio,categorie_ventre,categorie_torse,categorie_cuisses
2603,179.4002,56.0,109.315672,1.0,27.1018,68.1990,121.0056,131.3942,79.9084,95.3008,...,66.4972,113.8936,108.8898,26.7462,0.370664,0.634858,0.251593,rond,large,larges
4159,168.5036,58.3,75.600179,1.0,27.6098,62.9920,102.0064,121.3104,72.4916,82.0928,...,54.4068,94.3102,100.9904,24.2824,0.322882,0.559692,0.245855,rond,large,moyennes
2991,174.5996,32.0,78.471416,1.0,26.1112,66.8020,102.5906,124.7902,79.0956,86.1060,...,57.2008,85.5980,102.9970,27.2161,0.327611,0.490253,0.264329,moyen,large,moyennes
3753,174.3964,20.0,72.298029,1.0,26.3906,63.2968,95.4024,127.1016,81.0006,86.0044,...,57.5056,81.3054,102.4890,26.3271,0.329741,0.466210,0.213225,moyen,fin,moyennes
1865,181.5084,43.0,84.000702,1.0,27.6098,62.6110,105.9942,130.4544,86.5124,90.8050,...,61.7982,87.5030,106.2990,23.2664,0.340470,0.482088,0.216905,moyen,fin,moyennes


In [16]:
from sklearn.preprocessing import OneHotEncoder

# Colonnes à encoder
colonnes_cat = ['categorie_ventre', 'categorie_torse', 'categorie_cuisses']

# Afficher les valeurs uniques pour vérification
for col in colonnes_cat:
    print(f"Valeurs uniques pour '{col}' :", data[col].unique())

# Instancier OneHotEncoder avec drop='if_binary' (garde 1 seule colonne pour les colonnes binaires)
ohe = OneHotEncoder(sparse_output=False, drop='if_binary')

# Fit + transform sur les colonnes sélectionnées
encoded_array = ohe.fit_transform(data[colonnes_cat])

# Obtenir les noms des nouvelles colonnes générées
encoded_col_names = ohe.get_feature_names_out(colonnes_cat)

# Créer un DataFrame avec les colonnes encodées
df_encoded = pd.DataFrame(encoded_array, columns=encoded_col_names, index=data.index)

# Fusionner avec le DataFrame d’origine
mesure_sc = pd.concat([data.drop(columns=colonnes_cat), df_encoded], axis=1)

# Aperçu du DataFrame final
mesure_sc = mesure_sc.drop(columns=["sex"], errors='ignore')

mesure_sc.head()


Valeurs uniques pour 'categorie_ventre' : ['rond' 'moyen' 'plat']
Valeurs uniques pour 'categorie_torse' : ['large' 'fin' 'moyen']
Valeurs uniques pour 'categorie_cuisses' : ['larges' 'moyennes' 'fines']


,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,largeur_d_epaule_ratio,categorie_ventre_moyen,categorie_ventre_plat,categorie_ventre_rond,categorie_torse_fin,categorie_torse_large,categorie_torse_moyen,categorie_cuisses_fines,categorie_cuisses_larges,categorie_cuisses_moyennes
2603,179.4002,56.0,109.315672,27.1018,68.1990,121.0056,131.3942,79.9084,95.3008,51.4985,...,0.251593,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4159,168.5036,58.3,75.600179,27.6098,62.9920,102.0064,121.3104,72.4916,82.0928,45.9994,...,0.245855,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2991,174.5996,32.0,78.471416,26.1112,66.8020,102.5906,124.7902,79.0956,86.1060,48.5013,...,0.264329,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3753,174.3964,20.0,72.298029,26.3906,63.2968,95.4024,127.1016,81.0006,86.0044,47.7012,...,0.213225,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1865,181.5084,43.0,84.000702,27.6098,62.6110,105.9942,130.4544,86.5124,90.8050,49.6824,...,0.216905,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [17]:
from sklearn.preprocessing import RobustScaler

# 🟢 Colonnes à scaler (toutes les autres)
colonnes_a_scaler = ["taille", "age", "weight"]

# ⚙️ Instanciation et application du RobustScaler
scaler = RobustScaler()
df_scaled = mesure_sc.copy()
df_scaled[colonnes_a_scaler] = scaler.fit_transform(df_scaled[colonnes_a_scaler])
df_scaled.head()

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,largeur_d_epaule_ratio,categorie_ventre_moyen,categorie_ventre_plat,categorie_ventre_rond,categorie_torse_fin,categorie_torse_large,categorie_torse_moyen,categorie_cuisses_fines,categorie_cuisses_larges,categorie_cuisses_moyennes
2603,0.171625,0.913462,1.420392,27.1018,68.1990,121.0056,131.3942,79.9084,95.3008,51.4985,...,0.251593,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4159,-0.810069,1.024038,-0.216472,27.6098,62.9920,102.0064,121.3104,72.4916,82.0928,45.9994,...,0.245855,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2991,-0.260870,-0.240385,-0.077076,26.1112,66.8020,102.5906,124.7902,79.0956,86.1060,48.5013,...,0.264329,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3753,-0.279176,-0.817308,-0.376789,26.3906,63.2968,95.4024,127.1016,81.0006,86.0044,47.7012,...,0.213225,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1865,0.361556,0.288462,0.191368,27.6098,62.6110,105.9942,130.4544,86.5124,90.8050,49.6824,...,0.216905,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# Supprime toutes les colonnes contenant "ratio" dans leur nom
colonnes_ratio = [col for col in df_scaled.columns if "ratio" in col]
df_scaled.drop(columns=colonnes_ratio, inplace=True)

print(f"✅ Colonnes supprimées : {colonnes_ratio}")


✅ Colonnes supprimées : ['tour_de_cuisse_ratio', 'tour_de_taille_ratio', 'largeur_d_epaule_ratio']


In [20]:
from tpot import TPOTRegressor
from sklearn.impute import SimpleImputer
import pandas as pd
import os
import joblib

def entrainer_tpot_regressions_complet(df, colonnes_explicatives, colonnes_cibles, generations=4, population_size=20, random_state=42, save_dir='modeles_tpot_complet'):
    """
    Entraîne un modèle TPOTRegressor pour chaque cible sur l'ensemble du dataset (pas de split),
    puis sauvegarde le pipeline en .pkl.

    Args:
        df (pd.DataFrame): Données complètes.
        colonnes_explicatives (list): Colonnes d'entrée (features).
        colonnes_cibles (list): Colonnes cibles à prédire.
        generations (int): Nombre de générations TPOT.
        population_size (int): Taille de la population.
        random_state (int): Graine aléatoire pour reproductibilité.
        save_dir (str): Dossier où sauvegarder les fichiers .pkl.

    Returns:
        dict: Dictionnaire avec pipeline et chemin du modèle pour chaque cible.
    """
    os.makedirs(save_dir, exist_ok=True)
    resultats = {}

    # Imputation
    imputer = SimpleImputer(strategy='median')
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    for cible in colonnes_cibles:
        print(f"\n⚙️ Entraînement final complet pour : {cible}")

        X = df_imputed[colonnes_explicatives]
        y = df_imputed[cible]

        tpot = TPOTRegressor(
            generations=generations,
            population_size=population_size,
            verbose=2,
            random_state=random_state,
            n_jobs=1,
            cv=3  # ou 3 si tu veux aller plus vite
        )

        tpot.fit(X, y)

        # Sauvegarde
        chemin_fichier = os.path.join(save_dir, f"tpot_{cible}.pkl")
        joblib.dump(tpot.fitted_pipeline_, chemin_fichier)

        resultats[cible] = {
            'pipeline': tpot.fitted_pipeline_,
            'fichier_modele': chemin_fichier
        }

    return resultats

colonnes_X = ['taille', 'age', 'weight',
              'categorie_ventre_plat', 'categorie_ventre_moyen', 'categorie_ventre_rond',
              'categorie_torse_fin', 'categorie_torse_moyen', 'categorie_torse_large',
              'categorie_cuisses_fines', 'categorie_cuisses_moyennes', 'categorie_cuisses_larges']

# Vérifie que les colonnes existent dans df_scaled
colonnes_X_valides = [col for col in colonnes_X if col in df_scaled.columns]
colonnes_y = [col for col in df_scaled.columns if col not in colonnes_X_valides]

# Entraînement final sur tout le dataset
resultats_tpot_complet = entrainer_tpot_regressions_complet(df_scaled, colonnes_X_valides, colonnes_y)



⚙️ Entraînement final complet pour : tour_de_cheville


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52920 instead
  warnings.warn(
Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [15:05<00:00, 226.33s/it]
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warning


⚙️ Entraînement final complet pour : longueur_du_bras


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53122 instead
  warnings.warn(
Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [18:23<00:00, 275.81s/it]



⚙️ Entraînement final complet pour : tour_de_poitrine


...

c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55437 instead
  warnings.warn(
Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [20:27<00:00, 306.84s/it]
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti


⚙️ Entraînement final complet pour : hauteur_de_la_taille


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55725 instead
  warnings.warn(
Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [15:13<00:00, 228.39s/it]
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warning


⚙️ Entraînement final complet pour : longueur_d_avant_bras


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55894 instead
  warnings.warn(
Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [10:42<00:00, 160.72s/it]
